In [1]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

은닉층 - 신경망 성능 향상, 활성함수(2진분류-시그모이드) 등  

![image](https://user-images.githubusercontent.com/58851945/119298967-05e76100-bc99-11eb-9c4d-a2084768a2b4.png)

In [2]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

In [3]:
# 은닉층 - 활성화함수 담을때, 모델 성능 높일때 사용 
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 100개의 노드

# 출력층
dense2 = keras.layers.Dense(10, activation='softmax')  # hidden shape

# 심층신경망(DNN) 모델 - 은닉층을 가진 신경망
model = keras.Sequential([dense1, dense2])

은닉층(dense1)은 출력층(dense2) 보다 많아야한다
은닉층이 출력층보다 적으면 데이터가 손실될 수 있다
데이터 클래스가 10개라서 출력층에 10을 씀

층이 많아지면 name을 담기도 함
나중에 층을 쌓진않고 add라는 시퀀스 안에.... 뒤에 나옴
출력층은 제일 마지막에 와야 함

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


`Output Shape` - `None`  
epoch랑 비슷  
배치사이즈 - 한번에 32개의 노드를 읽어와서 실행(디폴트)  
한번 반복할때마다 실행시키는 노드의 수
  
`Param`  
파라미터의 수  
78500개의 픽셀에 100개의 가중치를 줌 <= (784+1)x100 = (784개의 픽셀+절편1개)x100  
100x10 + 10  
　　　　절편

모델 이름(name)은 영문자로 쓰는걸 권장(한글x)

###### 층을 추가하는 방법들
model = keras.Sequential([  
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),  
    keras.layers.Dense(10, activation='softmax', name='output')  
    ], name='fashion MNIST model')  

model = keras.Sequential()  
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape(784,)))  
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

In [6]:
# 모델 설정
model.compile(loss='sparse_categorical_crossentropy',  # 다중 분류
             metrics=['accuracy'])

# 모델 학습
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 61us/sample - loss: 0.5598 - accuracy: 0.8083
Epoch 2/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.4074 - accuracy: 0.8534
Epoch 3/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.3737 - accuracy: 0.8656
Epoch 4/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.3518 - accuracy: 0.8723
Epoch 5/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.3332 - accuracy: 0.8794


In [7]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

In [8]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28))) 
# Flatten 유틸리티층. (평평하게하다, 단조롭게하다) -> 직접 1차원으로 만들어줌
# input_shape 입력층 다음에 적어야 함

model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 55us/sample - loss: 0.5353 - accuracy: 0.8095
Epoch 2/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.3942 - accuracy: 0.8585
Epoch 3/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.3555 - accuracy: 0.8733
Epoch 4/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.3312 - accuracy: 0.8822
Epoch 5/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.3163 - accuracy: 0.8869


In [11]:
model.evaluate(val_scaled, val_target)

12000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.36222562243541084, 0.87733334]

SGD 확률적 경사하강법  Stochastic Gradient Descent
이동할 때 학습률(이동크기)를 조절 
적응적 학습률 : 거리가 먼 곳에선 step_size를 많이 주고, 가까운곳에선 학습률을 줄여서 step_size를 작게함  cf. 확률적 학습률
momentum 가속도(가속화). 디폴트 0

In [12]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [13]:
# 모델 설정 - adam
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.5214 - accuracy: 0.8175
Epoch 2/5
48000/48000 [==============================] - 2s 41us/sample - loss: 0.3949 - accuracy: 0.8582
Epoch 3/5
48000/48000 [==============================] - 2s 42us/sample - loss: 0.3547 - accuracy: 0.8714
Epoch 4/5
48000/48000 [==============================] - 2s 42us/sample - loss: 0.3301 - accuracy: 0.8779
Epoch 5/5
48000/48000 [==============================] - 2s 42us/sample - loss: 0.3114 - accuracy: 0.8848


In [14]:
model.evaluate(val_scaled, val_target)  # 검증된 정확도

12000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.3703820088803768, 0.8685]

In [15]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 설정 - RMSprop
model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 57us/sample - loss: 0.5303 - accuracy: 0.8120
Epoch 2/5
48000/48000 [==============================] - 2s 50us/sample - loss: 0.3932 - accuracy: 0.8592
Epoch 3/5
48000/48000 [==============================] - 2s 51us/sample - loss: 0.3564 - accuracy: 0.8709
Epoch 4/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.3367 - accuracy: 0.8788
Epoch 5/5
48000/48000 [==============================] - 2s 51us/sample - loss: 0.3199 - accuracy: 0.8864


In [16]:
(train_input, train_target), (test_input, test_target) = \
keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 
# train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = \
train_test_split(train_scaled, train_target, test_size=.2, random_state=42)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 모델 설정 - adagrad
model.compile(optimizer='adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Train on 48000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 56us/sample - loss: 0.7655 - accuracy: 0.7573
Epoch 2/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.6022 - accuracy: 0.8046
Epoch 3/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.5627 - accuracy: 0.8164
Epoch 4/5
48000/48000 [==============================] - 2s 49us/sample - loss: 0.5399 - accuracy: 0.8229
Epoch 5/5
48000/48000 [==============================] - 2s 48us/sample - loss: 0.5244 - accuracy: 0.8264
